In [ ]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../src"))

In [3]:
from detection.detector import Detector
from segmentation.segmenter import Segmenter
from utils import *

In [ ]:
damage_model = Detector('path/to/model')
parts_model = Segmenter('path/to/model')

summary = build_damage_summary('path/to/image', damage_model, parts_model, conf=0.3)

summary_for_report = []

for item in summary:
      
      print(f"{item['damage']} in {item['part']} "
            f"(conf={item['confidence']:.2f}, IoU={item['iou']:.2f})")
      
      summary_for_report.append(f"{item['damage']} in {item['part']} "
            f"(conf={item['confidence']:.2f}, IoU={item['iou']:.2f})")
      
summary_for_report

/opt/miniconda3/envs/yolo-mps/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


crack in front_bumper (conf=0.60, IoU=0.71)


['crack in front_bumper (conf=0.60, IoU=0.71)']

In [62]:
text_prepare(summary_for_report)

File created


In [ ]:
# Please install OpenAI SDK first: `pip3 install openai`
from openai import OpenAI
from dotenv import load_dotenv
from datetime import datetime

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

load_dotenv()

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'), base_url="https://api.deepseek.com")

with open("report/report.txt", "r", encoding="utf-8") as f:
    report_content = f.read()

user_data = {
    "make": "Toyota",
    "model": "Corolla",
    "year": "2018",
    "vin": "1234ABCD5678"
}

prompt = f"""
Generate a professional vehicle damage assessment report.
Include the report date, vehicle details, and the list of detected damages.

Each damage should be formatted like:
'<damage_type> in <vehicle_part> (conf=<confidence>, IoU=<IoU>) — Estimated repair cost: $XXX'

If any damage entry doesn't include a price, estimate a reasonable repair cost based on average market rates for that type of damage and part.

Report Date: {current_time}

Vehicle Information:
- Make: {user_data['make']}
- Model: {user_data['model']}
- Year: {user_data['year']}
- VIN: {user_data['vin']}

Detected Damages (from CV models):
{report_content}

Write the report in clear and structured English, ready to be converted into a PDF.
"""

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a report generator for vehicle damage assessments."},
        {"role": "user", "content": prompt}
    ],
    stream=False
)

print(response.choices[0].message.content)

# **Vehicle Damage Assessment Report**  
**Report Date:** August 7, 2025 | 13:56:03  

---  

## **Vehicle Information**  
- **Make:** Toyota  
- **Model:** Corolla  
- **Year:** 2018  
- **VIN:** 1234ABCD5678  

---  

## **Detected Damages**  
The following damages were identified using computer vision (CV) models. Confidence (conf) and Intersection over Union (IoU) scores indicate detection accuracy. Estimated repair costs are based on market averages.  

1. **Scratch in hood**  
   - **Detection Confidence:** 0.87  
   - **IoU Score:** 0.79  
   - **Estimated Repair Cost:** **$500.00** (Range: $425.00 – $575.00)  

2. **Dent in front door**  
   - **Detection Confidence:** 0.91  
   - **IoU Score:** 0.82  
   - **Estimated Repair Cost:** **$400.00** (Range: $340.00 – $460.00)  

3. **Crack in roof**  
   - **Detection Confidence:** 0.77  
   - **IoU Score:** 0.69  
   - **Estimated Repair Cost:** **$650.00** (LLM Estimate – Typical range: $550.00 – $750.00)  

---  

## **Total Est

In [55]:
text_prepare(response.choices[0].message.content)

File created


In [ ]:
damage = pd.read_csv('/Users/pacuk/Documents/cv-project/car-damage-assessment/data/damage_costs.csv')

# EXAMPLE

In [ ]:
report_lines = [
    "scratch in hood (conf=0.87, IoU=0.79)",
    "dent in front_door (conf=0.91, IoU=0.82)",
    "crack in roof (conf=0.77, IoU=0.69)",
]

user_data = {
    "make": "Toyota",
    "model": "Corolla",
    "year": "2018"
}

filtered = damage[
    (damage['make'] == user_data['make'].lower()) &
    (damage['model'] == user_data['model'].lower()) &
    (damage['model_year'] == int(user_data['year']))
]

fallback_means = (
    damage
    .groupby(['damage_type', 'part'])['average_cost_usd']
    .mean()
    .reset_index()
)


def get_estimated_cost(damage_type, part):
    match = filtered[
        (filtered['damage_type'] == damage_type) &
        (filtered['part'] == part)
    ]
    if not match.empty:
        avg = match['average_cost_usd'].mean()
        return round(avg, 2), round(avg * 0.85, 2), round(avg * 1.15, 2)

    fallback = fallback_means[
        (fallback_means['damage_type'] == damage_type) &
        (fallback_means['part'] == part)
    ]
    if not fallback.empty:
        avg = fallback['average_cost_usd'].values[0]
        return round(avg, 2), round(avg * 0.85, 2), round(avg * 1.15, 2)

    return None, None, None

final_damage_list = []

for line in report_lines:
    if " in " not in line:
        continue
    damage_type = line.split(" in ")[0].strip()
    part = line.split(" in ")[1].split(" ")[0].strip()
    est, low, high = get_estimated_cost(damage_type, part)
    if est:
        final_line = f"{line} — Estimated Cost: ${est} (range: ${low}–${high})"
    else:
        final_line = f"{line} — Estimated Cost: [LLM Estimate Needed]"
    final_damage_list.append(final_line)

final_damage_list


['scratch in hood (conf=0.87, IoU=0.79) — Estimated Cost: $500.0 (range: $425.0–$575.0)',
 'dent in front_door (conf=0.91, IoU=0.82) — Estimated Cost: $400.0 (range: $340.0–$460.0)',
 'crack in roof (conf=0.77, IoU=0.69) — Estimated Cost: [LLM Estimate Needed]']

In [56]:
text_prepare(final_damage_list)

File created
